In [98]:
# creates new directory with con image, t-maps, z-maps, and temporal BOLD 4D images
# and design files for OpenAutism (per subject, per task, per run)

# FOR FUTURE / PENDING:

# once second level is run, must import one con, t-map and z-map per subject (now importing 
# one per run)

# some subjects are in more than one experiment (e.g. SAX_OA_## in both exp1
# and exp2). In these cases, only one exp is copied over--the one where the subject has
# lower average framewise displacement. DEPENDING ON THE MIXED MODEL WE USE, WE MAY BE ABLE
# TO INCLUDE ALL RUNS ACROSS EXPERIMENTS FOR THESE SUBJECTS. BUT DOING THIS FOR NOW AS 
# PRECAUTIONARY MEASURE.

pilot = 1        # 1 = pilot_data / 0 = analysis_data

# import modules
from glob import glob
import re
import os.path
from itertools import repeat
import csv
from shutil import copyfile
import shutil
import random
import numpy as np
import pandas as pd
from collections import Counter

In [99]:
# subject look up table conversion (IGNORING undescores)
def Convert_Subname(Oldname):
    tmp_root = '/om/user/rezzo/Subject_Conversion_Table.csv'

    with open(tmp_root, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if Oldname == line[1].replace("_",""):
                Newname = line[0]
            else:
                continue
    return Newname  

In [100]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [101]:
def identify_greater_motion(subject1, subject2):
    tmp_root = '/om/user/rezzo/INCLUDE_final.csv'

    with open(tmp_root, "r") as tsv:
        mot1 = []
        mot2 = []
        subjectarray = [subject1, subject2]
        readCSV = csv.reader(tsv, delimiter='\t')
        for line in readCSV:
            if subject1 == line[2]:
                mot1.append(float(line[9]))
                np.array(mot1)
            if subject2 == line[2]:
                mot2.append(float(line[9]))
                np.array(mot2)
    if np.mean(mot1) > np.mean(mot2):
        return subject1
    elif np.mean(mot2) > np.mean(mot1):
        return subject2
    else:
        return random.choice(subjectarray) # temporarily missing data, just choose randomly

In [102]:
def remove_sub(name):
    if name[:4] == 'sub-':
        name = name[4:]
    return name

In [103]:
# Looks for subjects that participated in > 1 experiment and identifies which to exclude
# based on motion.

conversion = '/om/user/rezzo/Subject_Conversion_Table.csv'
df = pd.read_csv(conversion, header=None)

# make list of all duplicate items
a = df[0].tolist()
dup_list = [k for k,v in Counter(a).items() if v>1]

repeat_subjects = []
remove_subjects = []

for el in dup_list:
    with open(conversion, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if el == line[0]:
                repeat_subjects.append(line[1])

remove_from_list = [repeat_subjects[i:i + 2] for i in range(0, len(repeat_subjects), 2)]

for item in remove_from_list:
    fixname = identify_greater_motion('sub-'+item[0].replace("_",""), 'sub-'+item[1].replace("_",""))
    remove_subjects.append(remove_sub(fixname))

/opt/conda/envs/neuro/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/envs/neuro/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [104]:
root = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard.py/'

if pilot == 1:
    OA_dir = '/om/user/rezzo/OpenAutism/pilot_data/SUBJECTS/'
    remove_subjects = [] # to prevent the filtering out of any subjects in pilot_data
else:
    OA_dir = '/om/user/rezzo/OpenAutism/analysis_data/SUBJECTS/'

all_folders = glob(root+'*/') #obtain all folders within root directory
all_subjects = []

for folders in range(0, len(all_folders)):
    temp = os.listdir(all_folders[folders])
    for items in temp:
        all_subjects.append(all_folders[folders] +items)
        found = items.replace('sub-','')
        newname = Convert_Subname(found)

        if found in remove_subjects:
            print(found+" is a repeated subject with more motion in this round (skipping)")
        
        # create a folder with new name in the OA directory
        elif (not os.path.exists(OA_dir+newname)):  
            str1=re.split("/+", all_folders[folders])
            site = str1[-2]
            destroy = 0

            proc_streams = ['first_level_standard.py','first_level_aroma.py','first_level_gorgolewski.py']
            
            for stream in proc_streams:
                
                if destroy == 1:
                    break
                    
                stream_abrv = find_between(stream, "level_", ".py" )
                src_dir = '/om/group/saxelab/OpenAutism/Analysis/'+stream+'/'+site+'/sub-'+found+'/'
                
                # when raw data available per processing stream, specify below:
                design_dir = '/om/group/saxelab/OpenAutism/data/'+site+'/BIDS/'+'sub-'+found+'/func/'
                raw_dir = "/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/"+site+"/sub-"+found+"/"

                    
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'first_level_analyses','BOLD_data')) 
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','multivariate'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','lateralization'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','magnitude'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','interregional_cor'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','temporal_variance'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','MISC','individual_roi_masks'))
                os.makedirs(os.path.join(OA_dir+newname,'derivatives',stream_abrv,'second_level_analyses','MISC','mean_roi_Temporal_Signal'))


                # Copying over: CON, T-MAP, Z-STAT (excluding tomloc data)
                all_tasks = glob(src_dir+"*/")
                all_tasks = [tasks + 'model/' for tasks in all_tasks]

                # exclude tomloc data from analysis_data; or if pilot folder only include tomloc
                if pilot == 1:
                    all_tasks = [ task for task in all_tasks if "tomloc" in task ]
                else:
                    all_tasks = [ task for task in all_tasks if "tomloc" not in task ]
                
                # delete the subject folder in OA dir if no data (e.g. after removing tomloc)
                if all_tasks == []:
                    destroy = destroy +1
                    shutil.rmtree(OA_dir+newname)
                    break
                         
                for tasks in all_tasks:
                    str2=re.split("/+", tasks)
                    task_name = str2[-3]
                    all_runs = glob(tasks+"*/")
                    counter = 0
                    # put all_runs in order
                    all_runs = sorted(all_runs)
                    for runs in all_runs:   # just in case we end up separating runs within task
                        run_num = runs.split('/run')[-1][:-1]
                        run_num = 'run-00'+run_num
                        print(run_num)
                        prefixed1 = [filename for filename in os.listdir(runs) if (filename.startswith("con_1_") and filename.endswith("_tstat.nii.gz"))]
                        #print(runs+prefixed1[0])
                        copyfile(runs+prefixed1[0], OA_dir+newname+'/derivatives/'+stream_abrv+'/first_level_analyses/tstat1_'+task_name+'_'+run_num+'.nii.gz')
                        prefixed2 = [filename for filename in os.listdir(runs) if (filename.startswith("con_1_") and filename.endswith("_cope.nii.gz"))]
                        copyfile(runs+prefixed2[0], OA_dir+newname+'/derivatives/'+stream_abrv+'/first_level_analyses/cope1_'+task_name+'_'+run_num+'.nii.gz')
                        prefixed3 = [filename for filename in os.listdir(runs) if (filename.startswith("con_1_") and filename.endswith("_zstat.nii.gz"))]
                        copyfile(runs+prefixed3[0], OA_dir+newname+'/derivatives/'+stream_abrv+'/first_level_analyses/zstat1_'+task_name+'_'+run_num+'.nii.gz')


                    # Copying over: TEMPORAL DATA AND DESIGN FILE (analysis_data EXCLUDES TOMLOC DATA)
                    results = []
                        
                    for f in os.listdir(raw_dir): #tomloc/model/" # f is tasks
                        for mm in os.listdir(raw_dir+f+'/model/'):
                            run_num = mm.split('run')[1]
                            run_num = 'run-00'+run_num
                            for ll in os.listdir(raw_dir+f+'/model/'+mm):
                                if re.search(r'res4d.nii.gz', ll):
                                    results.append(f+'/'+ll)
                                    #print(f[-1])
                                    copyfile(raw_dir+f+'/model/'+mm+'/'+ll, OA_dir+newname+'/derivatives/'+stream_abrv+'/first_level_analyses/BOLD_data/temporaldata_'+f+'_'+run_num+'.nii.gz')

                    design = []  
                    for f in os.listdir(design_dir):
                        if (re.search(r'.tsv', f)):
                            design.append(f)

                    if pilot == 1:
                        design = [ N for N in design if "tomloc" in N ]
                    else:
                        design = [ N for N in design if "tomloc" not in N ]
                    
                    for elements in range(0, len(design)):   
                        abbrev = find_between( design[elements], "task-", "_events" )
                        copyfile(design_dir+design[elements], OA_dir+newname+'/derivatives/'+stream_abrv+'/first_level_analyses/BOLD_data/design_'+abbrev+'.tsv') 
                                      

run-001
run-002
run-003
run-004


FileNotFoundError: [Errno 2] No such file or directory: '/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/DOD/sub-SAXDOD025/'